# Analysis of Classification Models on Consumer Personality Analysis Dataset
Dataset: Consumer Personality Analysis (https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis/data)


## Objective
Using consumers' demographics, income, customer tenure, engagement, and spending behavior, classify them into 4 categories:
1. Loyalists: Long-term customers with high income and high spending
2. Potential Spenders: New customers with high income and high spending
3. At-Risk Customers: Long-term customers with significantly decreasing spending
4. Budget Customers: Customers with below-average income and low spending (mostly when discounts are available)

## Import Libraries and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics